# **Image Classification for Mushroom Species (K-Fold)**

## **Training the Model (Validation Accuracy and Loss)**

### **K-Fold Cross Validation**

In [ ]:
train_ds = train_ds.unbatch()

train_images = list(train_ds.map(lambda x, y: x))
train_labels = list(train_ds.map(lambda x, y: y))

val_ds = val_ds.unbatch()

val_images = list(val_ds.map(lambda x, y: x))
val_labels = list(val_ds.map(lambda x, y: y))

### **Merge Images and Labels**

In [ ]:
images = np.concatenate((train_images, val_images), axis=0)
labels = np.concatenate((train_labels, val_labels), axis=0)

### **Define the K-Fold and Model**

In [ ]:
from sklearn.model_selection import KFold

num_classes = len(class_names)

kfold = KFold(n_splits=10, shuffle=True)

fold_no = 1

for train, val in kfold.split(images, labels):

  model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
  ])

### **Compile the Model**

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### **Model Summary**

In [ ]:
model.summary()

### **Define the Callback**

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq="epoch")
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=40, verbose=1, mode='auto')

### **Train the Model**

In [ ]:
print('------------------------------------------------------------------------')
print(f'Training for fold {fold_no} ...')

acc_per_fold = []
loss_per_fold = []

epochs = 10
history = model.fit(
    images[val],
    labels[val],
    epochs=epochs,
    callbacks=[checkpoint, early],
)

scores = model.evaluate(images[val], labels[val], verbose=0)
print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
acc_per_fold.append(scores[1] * 100)
loss_per_fold.append(scores[0])

fold_no = fold_no + 1

### **Visualize Training Results**

In [ ]:
val_acc = history.history['val_accuracy']

val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Validation Loss')
plt.show()